In [ ]:
# Import required libraries
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Configure TensorFlow
tf.get_logger().setLevel('ERROR')
tf.random.set_seed(42)

# Constants
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS_NEW = 15
EPOCHS_FINE = 8
BASE_DIR = '/content/drive/MyDrive'
DATASET_PATH = f'{BASE_DIR}/dataset'
MODEL_PATH = f'{BASE_DIR}/plant_model.keras'
TFJS_PATH = f'{BASE_DIR}/plant_model_js'

# Create output directory
os.makedirs(TFJS_PATH, exist_ok=True)

# Enhanced data augmentation
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data generators
train_gen = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

num_classes = len(train_gen.class_indices)

def build_model(num_classes):
    # Initialize MobileNetV2 base
    base = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )
    base.trainable = False

    # Add custom layers
    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    # Create and compile model
    model = Model(base.input, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Load or create model
if os.path.exists(MODEL_PATH):
    print("Loading existing model...")
    model = load_model(MODEL_PATH)
    # Fine-tune last layers
    for layer in model.layers[-30:]:
        if not isinstance(layer, (Dropout, BatchNormalization)):
            layer.trainable = True
    epochs = EPOCHS_FINE
else:
    print("Creating new model...")
    model = build_model(num_classes)
    epochs = EPOCHS_NEW

# Enhanced callbacks
callbacks = [
    ModelCheckpoint(
        MODEL_PATH,
        save_best_only=True,
        monitor='val_accuracy',
        mode='max'
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3,
        min_lr=1e-6
    )
]

# Train model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    callbacks=callbacks,
    workers=4,
    use_multiprocessing=True
)

# Save model and convert to TensorFlow.js format
model.save(MODEL_PATH)
print(f"\nSaving model to {MODEL_PATH}")

# Convert to TF.js format
!rm -rf {TFJS_PATH}
os.makedirs(TFJS_PATH, exist_ok=True)
!tensorflowjs_converter --input_format=keras \
    --output_format=tfjs_layers_model \
    --quantize_uint8 \
    {MODEL_PATH} {TFJS_PATH}

# Save class labels
with open(os.path.join(TFJS_PATH, 'labels.js'), 'w') as f:
    f.write(f"const CLASS_NAMES = {list(train_gen.class_indices.keys())};")

# Print summary
print(f"\n✅ Model saved to: {MODEL_PATH}")
print(f"✅ TF.js model saved to: {TFJS_PATH}")
print(f"🧠 Trainable parameters: {sum(tf.keras.backend.count_params(w) for w in model.trainable_weights):,}")
model.summary()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install -q tensorflow tensorflowjs matplotlib
import os, tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

tf.get_logger().setLevel('ERROR')

BASE_DIR = '/content/drive/MyDrive'
DATASET_PATH = f'{BASE_DIR}/dataset'
MODEL_PATH   = f'{BASE_DIR}/plant_model.keras'
TFJS_PATH    = f'{BASE_DIR}/plant_model_js'

os.makedirs(TFJS_PATH, exist_ok=True)

In [ ]:

IMG_SIZE, BATCH, EPOCHS_NEW, EPOCHS_FINE = 224, 32, 10, 5

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    rotation_range=20, width_shift_range=0.1, height_shift_range=0.1,
    zoom_range=0.2, horizontal_flip=True
)

train_gen = datagen.flow_from_directory(DATASET_PATH, target_size=(IMG_SIZE, IMG_SIZE),
                                        batch_size=BATCH, class_mode='categorical',
                                        subset='training', shuffle=True)
val_gen = datagen.flow_from_directory(DATASET_PATH, target_size=(IMG_SIZE, IMG_SIZE),
                                      batch_size=BATCH, class_mode='categorical',
                                      subset='validation', shuffle=False)

num_classes = len(train_gen.class_indices)

In [ ]:
def build_model(num_classes):
    base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    base.trainable = False
    x = GlobalAveragePooling2D()(base.output)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    out = Dense(num_classes, activation='softmax')(x)
    model = Model(base.input, out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

if os.path.exists(MODEL_PATH):
    model = load_model(MODEL_PATH)
    for layer in model.layers[-20:]:
        if not isinstance(layer, Dropout): layer.trainable = True
    epochs = EPOCHS_FINE
else:
    model = build_model(num_classes)
    epochs = EPOCHS_NEW

callbacks = [
    ModelCheckpoint(MODEL_PATH, save_best_only=True, monitor='val_accuracy', mode='max'),
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
]

history = model.fit(train_gen, validation_data=val_gen, epochs=epochs, callbacks=callbacks)

Epoch 1/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 658s 3s/step - accuracy: 0.9993 - loss: 0.0035 - val_accuracy: 0.9969 - val_loss: 0.0082
Epoch 2/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 586s 2s/step - accuracy: 0.9987 - loss: 0.0036 - val_accuracy: 0.9963 - val_loss: 0.0140
Epoch 3/5
 43/240 ━━━━━━━━━━━━━━━━━━━━ 6:33 2s/step - accuracy: 0.9975 - loss: 0.0066

In [ ]:
model.save(MODEL_PATH)
!rm -rf {TFJS_PATH}
os.makedirs(TFJS_PATH, exist_ok=True)
!tensorflowjs_converter --input_format=keras --output_format=tfjs_layers_model {MODEL_PATH} {TFJS_PATH}
with open(os.path.join(TFJS_PATH, 'labels.js'), 'w') as f:
    f.write(f"const CLASS_NAMES = {list(train_gen.class_indices.keys())};")

print(f"✅ MODEL: {MODEL_PATH}")
print(f"✅ TFJS:  {TFJS_PATH}")
print(f"🧠 Trainable params: {sum(tf.keras.backend.count_params(w) for w in model.trainable_weights):,}")
model.summary()
